In [1]:
import os
os.chdir("../")

import semiolog as slg

In [2]:
import string
from collections import Counter,defaultdict
from sklearn.cluster import KMeans
from sklearn.preprocessing import normalize
import numpy as np
import seaborn as sns
from itertools import compress
from scipy import sparse

In [3]:
sns.set(rc = {'figure.figsize':(30,4)})
def plot(vector):
    return sns.heatmap(vector, xticklabels=elements[:-1])#,yticklabels=elements[:-1])

In [79]:
level = "words"

if level == "tags":
    ng1 = slg.util.load_file("wip/res_bnc/ng_t_1.json")
    ng2 = slg.util.load_file("wip/res_bnc/ng_t_2.json")
    ng3 = slg.util.load_file("wip/res_bnc/ng_t_3.json")
    ng4 = slg.util.load_file("wip/res_bnc/ng_t_4.json")

elif level == "words":
    ng1 = slg.util.load_file("wip/res_bnc/ng_w_1.json")
    ng2 = slg.util.load_file("wip/res_bnc/ng_w_2.json")
    ng3 = slg.util.load_file("wip/res_bnc/ng_w_3_5.json")
    ng4 = slg.util.load_file("wip/res_bnc/ng_w_4_5.json")

elif level == "chars":
    ng1 = slg.util.load_file("models/en_bnc/vocabulary/alpha.json")
    ng2 = slg.util.load_file("models/en_bnc/vocabulary/ngrams_rem/big_ng/2.json")
    ng3 = slg.util.load_file("models/en_bnc/vocabulary/ngrams_rem/big_ng/3.json")
    ng4 = slg.util.load_file("models/en_bnc/vocabulary/ngrams_rem/big_ng/4.json")

In [80]:
thres = 100

if level == "chars":

    ng1 = {k:v for k,v in ng1.items() if v>=thres}
    ng2 = {tuple(list(k)):v for k,v in ng2.items() if v>=thres}
    ng3 = {tuple(list(k)):v for k,v in ng3.items() if v>=thres}
    ng4 = {tuple(list(k)):v for k,v in ng4.items() if v>=thres}
    
elif level == "words":
    ng2 = {tuple(k.split()):v for k,v in ng2.items() if v>=thres}
    ng3 = {tuple(k.split()):v for k,v in ng3.items() if v>=thres}
    ng4 = {tuple(k.split()):v for k,v in ng4.items() if v>=thres}

    ng2 = {k:v for k,v in ng2.items() if len(k)>1 and all([kn not in ",.'‘’?!"+string.punctuation for kn in k])}
    ng3 = {k:v for k,v in ng3.items() if len(k)>2 and all([kn not in ",.'‘’?!"+string.punctuation for kn in k])}
    ng4 = {k:v for k,v in ng4.items() if len(k)>3 and all([kn not in ",.'‘’?!"+string.punctuation for kn in k])}

else:    
    ng2 = {tuple(k.split()):v for k,v in ng2.items() if v>=thres}
    ng3 = {tuple(k.split()):v for k,v in ng3.items() if v>=thres}
    ng4 = {tuple(k.split()):v for k,v in ng4.items() if v>=thres}

In [81]:
if level == "chars":
    elements = string.ascii_lowercase+string.digits
    
else:
    elements = ng1.keys()

if level == "words":
    elements = [e for e in elements if e not in ",.'‘’?!"+string.punctuation]#[:100000]

elements = list(elements) + [" "]
elements_dict = {k:i for i,k in enumerate(elements)}
elements_len = len(elements)

In [92]:
term = "that"

contexts = {(cl,cr):v for (cl,t,cr),v in ng3.items() if t == term and cl in elements and cr in elements}

terms = defaultdict(int)
for (cl,t,cr),v in ng3.items():
    if (cl,cr) in contexts:
        terms[t] += v

len(contexts)

831

In [93]:
ortho = {(c,(l,r)):v for (l,c,r),v in ng3.items()}

a_M = slg.util.build_term_context_matrix(elements, contexts, ortho)

Building oR Matrix...
Term-Context Matrix built in 177.26 secs.



In [94]:
contexts_kmeans = KMeans(n_clusters=4, random_state=0).fit(normalize(a_M.T,norm="l1",axis=1))

In [95]:
for c_n in range(len(contexts_kmeans.cluster_centers_)):
    cluster_n = contexts_kmeans.cluster_centers_[c_n]

    D = sparse.diags(cluster_n)
    contexts_full = D.dot(a_M)
    contexts_norm = normalize(np.asarray(contexts_full.sum(axis=0)),norm="l1")
    print(contexts_kmeans.cluster_centers_[:,elements_dict[term]][c_n])
    print([list(contexts.keys())[i] for i in  np.flip(np.argsort(contexts_norm))[0][:20] if contexts_norm[0][i]>0])
    print([elements[i] for i in  np.flip(np.argsort(cluster_n))[:20] if cluster_n[i]>0])
    print()
    # plot(contexts_kmeans.cluster_centers_[c_n:c_n+1])

0.11406209403790801
[('at', 'time'), ('in', 'case'), ('at', 'moment'), ('in', 'way'), ('of', 'year'), ('by', 'time'), ('in', 'country'), ('all', 'time'), ('on', 'basis'), ('in', 'area'), ('in', 'context'), ('of', 'time'), ('in', 'sense'), ('is', 'most'), ('in', 'year'), ('on', 'one'), ('to', 'point'), ('to', 'end'), ('at', 'age'), ('is', 'the')]
['the', 'this', 'that', 'a', 'any', 'one', 'some', 'only', 'which', 'an', 'no', 'his', 'either', 'each', 'just', 'all', 'another', 'last', 'its', 'what']

0.9741279761258902
[('is', 'the'), ('and', "'s"), ('so', 'the'), ('and', 'the'), ('fact', 'the'), ('is', 'it'), ('at', 'time'), ('said', 'the'), ('well', "'s"), ('and', 'was'), ('but', "'s"), ('and', 'is'), ('ensure', 'the'), ('so', 'they'), ('think', "'s"), ('so', 'it'), ('so', "'s"), ('was', 'the'), ('is', 'they'), ('to', 'of')]
['that', 'in', 'of', 'what', 'as', 'by', 'when', 'to', 'on', 'like', 'only', 'why', 'about', 'which', 'and', 'this', 'for', 'out', 'how', 'not']

0.1820552981448885

In [234]:
cluster_n = 3

In [21]:
context_cluster = defaultdict(list)
for i,c in enumerate(contexts_kmeans.labels_):
    context_cluster[c] += [i]

In [27]:
terms_clusters = [[elements[sort] for sort in np.argsort(centroid)] for centroid in contexts_kmeans.cluster_centers_]

In [30]:
terms_clusters[2][:20]

[' ',
 '2',
 '1',
 '0',
 'z',
 'v',
 '9',
 '5',
 'q',
 '3',
 '6',
 '4',
 'j',
 '8',
 '7',
 'h',
 'k',
 'b',
 'g',
 'w']

In [96]:
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.datasets import make_multilabel_classification
# This produces a feature matrix of token counts, similar to what
# CountVectorizer would produce on text.
X, _ = make_multilabel_classification(random_state=0)
lda = LatentDirichletAllocation(n_components=5,
    random_state=0)
lda.fit(X)

# get topics for some given samples:
lda.transform(X[-2:])

array([[0.00360392, 0.25499205, 0.0036211 , 0.64236448, 0.09541846],
       [0.15297572, 0.00362644, 0.44412786, 0.39568399, 0.003586  ]])

In [97]:
X

array([[3., 1., 4., ..., 4., 1., 3.],
       [5., 0., 6., ..., 0., 0., 3.],
       [3., 4., 1., ..., 3., 2., 5.],
       ...,
       [2., 1., 2., ..., 1., 0., 3.],
       [6., 4., 1., ..., 1., 3., 5.],
       [2., 4., 2., ..., 5., 4., 2.]])